### Реранкинг и визуализация

In [ ]:
import re
import spacy
import folium

from openai import OpenAI
from yaml import safe_load
from django.shortcuts import render
from .forms import MeasurementModelForm
from .utils import get_center_coordinates
from sentence_transformers import CrossEncoder
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings

Загружаем модель для русского языка `ru_core_news_sm`. Это предварительно обученная модель для обработки текста на русском языке, которая предоставляет различные возможности для анализа текста, такие как выделение сущностей (например, имен, дат), части речи, лемматизацию и т.д.

In [ ]:
nlp = spacy.load("ru_core_news_sm")
with open('geo.yml', 'r') as f:
  data = safe_load(f)

Устанавливаем количество заведений в выдаче.

In [ ]:
COUNT = 3

Инициализируем компоненты системы для работы с векторами. Для реранкинга будем использовать `cross-encoder-russian-msmarco` — модель, которая используется для обработки текста на русском языке. Она была обучена на данных с поисковыми запросами и отзывами, что позволяет ей эффективно выполнять задачи, связанные с ранжированием текстов (например, сортировка отзывов по релевантности). Модель использует архитектуру cross-encoder, которая представляет собой двустороннюю нейронную сеть, принимающую пару текстов и оценивающую их взаимодействие, а не просто их представления.

In [ ]:
embedding_function = SentenceTransformerEmbeddings(
    model_name="deepvk/USER-bge-m3"
)
cross_model = CrossEncoder(
    'cross-encoder-russian-msmarco', 
    max_length=512
)
db1 = Chroma(
    persist_directory="./db/", 
    embedding_function=embedding_function, 
    collection_name="review_flamp_yandex_v7_exploaded__USER-bge-m3"
)

Инициализируем функцию для очистки текста. Будем использовать её, чтоб очистить запрос перед тем, как отправить на поиск в базе данных и для очистки текста на передачу в систему для генерации ответа через OpenAI API.

In [ ]:
def clean_text(text):
  try:
    text = text.lower() 
    text = re.sub(r"[^\w\s\n]", " ", text) 
    text = re.sub(r'[^а-яА-Яa-zA-Z]', ' ', text) 

    doc = nlp(text)
    lemmas = [token.lemma_ for token in doc]
    text = " ".join(lemmas)

    doc = nlp(text)
    filtered_tokens = [token.text for token in doc if not token.is_stop]
    text = " ".join(filtered_tokens)

  except:
      text = ""

  return text

Инициализируем функцию, которая будет отвечать за взаимодействие с Chat GPT.

In [ ]:
def process_text(user_input, places):
    system_prompt = f""" 
    Ты хочешь помочь клиенту найти наилучшее заведение. Система выбрала 3 ресторана по отзывам, разнообразь
    ответ для клиента.
    Если клиент не написал ничего, что относится к поиску ресторана или еде, то ответь '-'.
    Ниже будут предоставлены отзывы о заведениях и запрос от клиента.
    - Коротко напиши интересный факт или шутку  том, что ищет клиент.
    - Коротко напиши обзор о заведениях на основе отзывов.
    Заведения начинаются с 'Название'.
    Если клиент не написал ничего, что относится к поиску ресторана или еде, то ответь '-'.
    Отзывы: {places}
    """ 

    client = OpenAI(
    api_key=data["PROXY_API_KEY"],
    base_url=data["PROXY_API_URL"],
)
    completion = client.chat.completions.create(
        model="gpt-4o-mini",  
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input},
        ],
        max_tokens=300, 
        temperature=0.3, 
    )
    response = completion.choices[0].message
    answer = response.content
    return answer

Инициализируем функцию обработки запросов, которая выполняет следуюшие действия:
- Получает запрос пользователя и ищет релевантные заведения.
- Пересортировывает результаты с помощью модели.
- Отображает результаты на карте с маркерами.
- Генерирует ответ на основе отзывов о заведениях и выводит его пользователю.

In [ ]:
def process_query(request):
    form = MeasurementModelForm(request.POST or None)
    
    # координаты центра Москвы
    # TODO определение координат пользователя
    location_lat, location_lon = (55.751477, 37.619003)
    #инициируем карту    
    m = folium.Map(
        location=get_center_coordinates(location_lat, location_lon), 
        zoom_start=10
    )
    output = "Интересный факт"

    if form.is_valid():
        query = form.cleaned_data.get('destination')
        # 1-й шаг. Поиск с использованием биэнкодера
        docs1 = db1.similarity_search_with_relevance_scores(clean_text(query), k=10)
        # 2-й шаг. Реранкинг
        scores = cross_model.predict(
            [(f"{query} - хороший отзыв", doc[0].metadata["review"]) for doc in docs1]) 
        for i in range(len(scores)):
            docs1[i] = (*docs1[i], scores[i])
        # Пересортируем результаты, с учетом данных от реранкера
        docs = sorted(docs1, key=lambda p: p[2], reverse=True)
        k=0
        places = ""
        names = []
        for d in docs:
            if k == COUNT:
                break
            marker = folium.Marker(
                location=[d[0].metadata["lat"], d[0].metadata["lon"]],
                tooltip=f'{d[0].metadata["name"]} {d[0].metadata["rating"]}',
                popup=d[0].page_content,
                icon=folium.Icon(color='red', icon='cloud')
            )
            marker.add_to(m)
            places+=f"Название {d[0].metadata['name']}. Отзыв {d[0].metadata['review']} \n"
            if not d[0].metadata["name"] in names:
                names.append(d[0].metadata["name"])
                k+=1
        output = process_text(query, places)
    
    #рендерим карту в html
    m = m._repr_html_()
    
    # TODO определение расстояния от пользователя до найденных точек
    context = {
        'distance': 0,
        'destination': 0,
        'form': form,
        'map':m,
        'text': output
    }
    
    return render(request, 'measurements/main.html', context)